In [1]:
# Importing required modules

import pandas as pd
import numpy as np

In [2]:
# Loading raw dataset and renaming columns

df = pd.read_csv("pollution.csv")
df.columns = ['date', 'year', 'month', 'day', 'address', 'state', 'county', 'city', 'o3_mean', 'o3_max_value', 'o3_max_hour', 'o3_aqi', 'co_mean', 'co_max_value', 'co_max_hour', 'co_aqi', 'so2_mean', 'so2_max_value', 'so2_max_hour', 'so2_aqi', 'no2_mean', 'no2_max_value', 'no2_max_hour', 'no2_aqi']

In [3]:
# Creating date and location dimension tables

date_df = df[['date', 'year', 'month', 'day']]
location_df = df[['address', 'state', 'county', 'city']]

In [4]:
# Dropping duplicates in date table and setting date_id for PK and FK reference

date_df = date_df.drop_duplicates()
date_df['date_id'] = np.arange(1, len(date_df)+1)
dates = {d.date: d.date_id for i, d in date_df.iterrows()}
date_df['date'] = pd.to_datetime(df['date'])

In [5]:
# Dropping duplicates in location table and setting location_id for PK and FK reference

location_df = location_df.drop_duplicates()
location_df['location_id'] = np.arange(1, len(location_df)+1)
locations = {}
for i, d in location_df.iterrows():
    tup = tuple([d.address, d.state, d.county, d.city])
    locations[tup] = d.location_id

In [6]:
# Creating fact table with pollution data

fact_cols = ['date', 'address', 'state', 'county', 'city', 'o3_mean', 'o3_max_value', 'o3_max_hour', 'o3_aqi', 'co_mean', 'co_max_value', 'co_max_hour', 'co_aqi', 'so2_mean', 'so2_max_value', 'so2_max_hour', 'so2_aqi', 'no2_mean', 'no2_max_value', 'no2_max_hour', 'no2_aqi']
fact_df = df[fact_cols]

In [7]:
fact_df['date_id'] = fact_df.date.apply(lambda x: dates[x])

C:\Users\deva\AppData\Local\Temp\ipykernel_18588\3560010315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_df['date_id'] = fact_df.date.apply(lambda x: dates[x])


In [8]:
a = []
for i, d in fact_df.iterrows():
    tup = tuple([d.address, d.state, d.county, d.city])
    a.append(locations[tup])

In [9]:
fact_df['location_id'] = np.array(a)

C:\Users\deva\AppData\Local\Temp\ipykernel_18588\1894522726.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_df['location_id'] = np.array(a)


Saving the processed data into 3 csv files.

In [15]:
date_df.to_csv("dates_dim.csv", index=False)
location_df.to_csv("location_dim.csv", index=False)
fact_df.to_csv("pollution_fact.csv", index=False)